In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [2]:
!pip install transformers -q
!pip install -q -U bitsandbytes>=0.44.0
!pip install rouge-score -q

  Preparing metadata (setup.py) ... done


In [3]:
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.8 MB/s eta 0:00:00


In [4]:
PATH_TO_DATA = '/content/drive/MyDrive/cleaned_api_bank_data.xlsx'
MODEL = "microsoft/Phi-3-mini-4k-instruct"#"microsoft/phi-2"# "microsoft/phi-2"
LOAD_IN_4_BIT = True
SAMPLE = None


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
import re
from rouge_score import rouge_scorer


In [6]:
from tqdm.auto import tqdm
tqdm.pandas()


api_bank_df = pd.read_excel(PATH_TO_DATA)#encoding='utf8', engine='python',on_bad_lines="warn"
api_bank_df.split.value_counts()

test_results = api_bank_df[api_bank_df['split']=='test']
if SAMPLE:
  test_results = test_results.sample(SAMPLE)
else:
  print("No sampling!")

No sampling!


In [7]:
import huggingface_hub
from google.colab import userdata

HF_WRITE_KEY = userdata.get('HF_WRITE_KEY')

huggingface_hub.login(token=HF_WRITE_KEY)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=LOAD_IN_4_BIT,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
     bnb_4bit_use_double_quant=True
)

tokeniser = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForCausalLM.from_pretrained(MODEL,# attn_implementation="flash_attention_2",
                                                #  quantization_config = ...
                                             quantization_config=bnb_config,
                                             trust_remote_code=True
                                             )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/940 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [9]:
def format_prompt(prompt):
    text = f"<|user|>\n {prompt} <|end|>\n<|assistant|>"
    return text


def generate_completion(input_str, max_new_tokens):
  inputs = tokeniser(input_str, return_tensors="pt")

  outputs = model.generate(input_ids=inputs['input_ids'],max_new_tokens=max_new_tokens,do_sample=True,temperature=0.01)

  prompt_length = inputs['input_ids'].shape[1]

  answer = tokeniser.decode(outputs[0][prompt_length:])

  return answer


  # api_bank_df.head(1)

def generate_model_answers(test_results, model_object, model_name, tokeniser=tokeniser):

    test_results['answer_tokens'] = test_results['function_calls'].apply(lambda x: len(tokeniser.encode(x)))
    test_results['prompt_tokens'] = test_results['prompt'].apply(lambda x: len(tokeniser.encode(x)))

    test_results['formatted_prompt'] = test_results['prompt'].apply(lambda x: format_prompt(x))
    test_results['model_answer'] = test_results.progress_apply(lambda x: generate_completion(x['formatted_prompt'], max_new_tokens=x['answer_tokens']+20) if x['split'] == 'test' else None, axis=1)

    test_results['model'] = model_name


    return test_results

test_results_copy = test_results.copy()
test_results = generate_model_answers(test_results, model_object=model,model_name=MODEL,tokeniser=tokeniser)



<ipython-input-9-ca809b934684>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_results['answer_tokens'] = test_results['function_calls'].apply(lambda x: len(tokeniser.encode(x)))
<ipython-input-9-ca809b934684>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_results['prompt_tokens'] = test_results['prompt'].apply(lambda x: len(tokeniser.encode(x)))
<ipython-input-9-ca809b934684>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

  0%|          | 0/997 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
<ipython-input-9-ca809b934684>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_results['model_answer'] = test_results.progress_apply(lambda x: generate_completion(x['formatted_prompt'], max_new_tokens=x['answer_tokens']+20) if x['split'] == 'test' else None, axi

In [10]:

test_results['completion']=test_results['completion'].astype(str).str.strip()
test_results['function_call']=test_results['function_call'].astype(str).str.strip()
test_results['function_name']=test_results['function_name'].astype(str).str.strip()

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
# scores =
def evaluate_test_results(df,rouge_scorer=scorer):

  test_results = df.copy()
  test_results['contained_correct_call'] = test_results.apply(lambda x: x['completion'] in x['model_answer'], axis=1)
  test_results['exact_match'] = test_results.apply(lambda x: x['function_call'] == x['model_answer'] if x['model_answer'] else None, axis=1)
  test_results['contains_function_name'] = test_results.apply(lambda x: x['function_name'] in x['model_answer'] if x['model_answer'] else None, axis=1)
  test_results['answer_included_list'] = test_results.apply(lambda x: bool(re.search(r'\[.*\]', x['model_answer'])) if x['model_answer'] else None, axis=1)
  test_results['answer_list'] = test_results.apply(lambda x: re.search(r'\[.*\]', x['model_answer']).group(0) if x['answer_included_list'] else '', axis=1)



  test_results['rouge_1_score'] = test_results.apply(lambda x: rouge_scorer.score(target=x['completion'],prediction=x['answer_list']),axis=1)

  test_results['rouge_1_precision_list'] = test_results['rouge_1_score'].apply(lambda x: x['rouge1'].precision)
  test_results['rouge_1_recall_list'] =  test_results['rouge_1_score'].apply(lambda x: x['rouge1'].recall)
  test_results['rouge_1_f1_list'] =  test_results['rouge_1_score'].apply(lambda x: x['rouge1'].fmeasure)


  return test_results

evaluated_results = evaluate_test_results(test_results)
evaluated_results[['contained_correct_call','exact_match','answer_included_list','contains_function_name','rouge_1_precision_list','rouge_1_recall_list','rouge_1_f1_list']].mean()

<ipython-input-10-4f549b3f0c9c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_results['completion']=test_results['completion'].astype(str).str.strip()
<ipython-input-10-4f549b3f0c9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_results['function_call']=test_results['function_call'].astype(str).str.strip()
<ipython-input-10-4f549b3f0c9c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

contained_correct_call    0.281846
exact_match               0.000000
answer_included_list      0.935807
contains_function_name    0.701103
rouge_1_precision_list    0.518666
rouge_1_recall_list       0.579224
rouge_1_f1_list           0.533090
dtype: float64

In [ ]:
OUTPUT_PATH = f"/content/drive/MyDrive/api_bank_results_{MODEL.split('/')[-1]}.xlsx".replace("-","_")

evaluated_results.to_excel(OUTPUT_PATH)

In [ ]:
from google.colab import runtime
runtime.unassign()